<a href="https://colab.research.google.com/github/eamirhn/Kaggle-Projects/blob/main/ChatBot_Medical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install gpt4all

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00


In [2]:
from gpt4all import GPT4All
model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf") # downloads / loads a 4.66GB LLM

Downloading: 100%|██████████| 4.66G/4.66G [01:18<00:00, 59.4MiB/s]
Verifying: 100%|██████████| 4.66G/4.66G [00:14<00:00, 330MiB/s]


In [ ]:
text = """
Analyze the following radiomic features of a brain tumor and provide a brief explanation:
    'original_shape_Elongation': 0.7532,
    'original_firstorder_Energy': 1256789.3245,
    'original_glcm_Correlation': 0.9876,
"""
with model.chat_session():
    print(model.generate(f"Are you Famillier with brian tumors? {text}", max_tokens=210))

A fascinating topic!

Yes, I'm familiar with brain tumors and radiomic features.

Let's break down the three radiomic features you provided:

1. **Original_Shape_Elongation**: This feature measures the elongation of a tumor shape in its original form (i.e., before any processing or segmentation). Elongation is defined as the ratio of the longest axis to the shortest axis of an object. In this case, the value 0.7532 indicates that the brain tumor has an elongated shape with a relatively long major axis compared to its minor axes.
2. **Original_Firstorder_Energy**: This feature represents the first-order energy moment of the tumor's original intensity histogram. Energy is a measure of the total power or magnitude of the signal, and in this case, it's extremely high (1256789.3245). A higher energy value can indicate that the tumor has a complex texture with many different gray levels.
3. **Original_GLCM_Correlation**: This feature measures the correlation between neighboring pixels


In [ ]:
with model.chat_session():
    print(model.generate("What are the key features used in brain tumor radiomics?", max_tokens=210))

Brain tumor radiomics is a rapidly evolving field that aims to extract meaningful information from medical images, such as MRI and CT scans, to improve diagnosis, prognosis, and treatment of brain tumors. The following are some of the key features commonly used in brain tumor radiomics:

1. **Texture Features**: These describe the spatial arrangement of pixels or voxels within an image. Examples include:
	* Coarseness: measures the size of texture elements.
	* Contrast: measures the difference between light and dark areas.
	* Homogeneity: measures how uniform the texture is.
2. **Shape Features**: These describe the morphology of a tumor, such as its shape, size, and orientation. Examples include:
	* Elliptical fit: measures the degree to which a tumor's shape can be approximated by an ellipse.
	* Sphericity: measures the ratio of a tumor's volume to its surface area.
3. **Intensity Features**: These describe the intensity values within an image, such as mean and standard deviation. Ex

# RAG

In [4]:
!pip -q install gpt4all langchain faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 75.1 MB/s eta 0:00:00


In [12]:
!pip -q install -U langchain-community


In [11]:
!pip install -q gpt4all langchain faiss-cpu
from gpt4all import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import GPT4All as LangChainGPT4All
from langchain.document_loaders import TextLoader

# Load and process documents
docs = []
for file in ["/content/drive/MyDrive/RAG/doc1_original_shape_Elongation.txt",
              "/content/drive/MyDrive/RAG/doc2_original_firstorder_Energy.txt",
              "/content/drive/MyDrive/RAG/doc3_original_glcm_Correlation.txt"]:
    loader = TextLoader(file)
    docs.extend(loader.load())

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create embeddings and vector store
embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(splits, embeddings)

# Initialize GPT4All model
model = LangChainGPT4All(model="Meta-Llama-3-8B-Instruct.Q4_0.gguf")

# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# Function to generate response
def generate_response(query):
    return qa_chain.run(query)

<ipython-input-11-e20624ddb927>:23: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [13]:
text = """
Analyze the following radiomic features of a brain tumor and provide a brief explanation:
    'original_shape_Elongation': 0.7532,
    'original_firstorder_Energy': 1256789.3245,
    'original_glcm_Correlation': 0.9876,
"""
query = f"Are you familiar with brain tumors? {text}"
response = generate_response(query)
print(response)

<ipython-input-11-e20624ddb927>:38: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return qa_chain.run(query)


 
I am familiar with brain tumors.

The provided radiomic features are:

1. 'original_shape_Elongation': This feature measures the elongation of a tumor, which is defined as the ratio between the major axis length and the minor axis length of the bounding box. A value closer to 1 indicates a more elongated shape, while a value closer to 0 suggests a shape closer to a sphere.

Value: 0.7532

This relatively high value (closer to 1) may indicate that the tumor has an irregular or elongated shape, which could be associated with certain types of brain tumors.

2. 'original_firstorder_Energy': This feature quantifies the sum of squared voxel intensities within a defined region of interest (ROI), such as a brain tumor. Higher energy values can reflect regions with dense cellularity or increased contrast enhancement, often seen in high-grade tumors.

Value: 1256789.3245

This extremely high value may suggest that the tumor has an intense signal intensity and heterogeneity, which could be indi

In [ ]:
import faiss
import pickle

# Assuming 'vectorstore' is your FAISS object
faiss.write_index(vectorstore.index, "vector_store.index")

# Save any additional metadata if necessary
with open("vector_store_metadata.pkl", "wb") as f:
    pickle.dump(vectorstore.metadata, f)


In [ ]:
import faiss
import pickle

# Load the FAISS index
index = faiss.read_index("vector_store.index")

# Load any additional metadata if necessary
with open("vector_store_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

# Create a new vector store using the loaded index
vectorstore = FAISS(index=index, metadata=metadata)


In [14]:
import time

# Assuming you've already set up the RAG system as in the previous code

# Function to generate response with timing
def generate_timed_response(query):
    start_time = time.time()
    response = qa_chain.run(query)
    end_time = time.time()
    processing_time = end_time - start_time
    return response, processing_time

# Example usage
query = """Analyze the following radiomic features of a brain tumor and provide a brief explanation:
    'original_shape_Elongation': 0.7532,
    'original_firstorder_Energy': 1256789.3245,
    'original_glcm_Correlation': 0.9876"""

response, processing_time = generate_timed_response(query)

print(f"Query: {query}\n")
print(f"Response: {response}\n")
print(f"Processing time: {processing_time:.2f} seconds")

# You can now easily query the system multiple times
queries = [
    "What are the common symptoms of brain tumors?",
    "How does tumor shape affect prognosis?",
    "Explain the significance of GLCM correlation in tumor analysis."
]

for query in queries:
    response, processing_time = generate_timed_response(query)
    print(f"\nQuery: {query}")
    print(f"Response: {response}")
    print(f"Processing time: {processing_time:.2f} seconds")


Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7a60f781a5f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gpt4all/_pyllmodel.py", line 614, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 


Query: Analyze the following radiomic features of a brain tumor and provide a brief explanation:
    'original_shape_Elongation': 0.7532,
    'original_firstorder_Energy': 1256789.3245,
    'original_glcm_Correlation': 0.9876

Response: 

Processing time: 296.66 seconds


Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7a60f781a5f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gpt4all/_pyllmodel.py", line 614, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 



Query: What are the common symptoms of brain tumors?
Response: 
Processing time: 12.75 seconds


Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7a60f781a5f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gpt4all/_pyllmodel.py", line 614, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 



Query: How does tumor shape affect prognosis?
Response: 
Processing time: 146.75 seconds

Query: Explain the significance of GLCM correlation in tumor analysis.
Response:  The original_ glcm_Correlation feature measures the linear dependency of gray-level values within a tumor ROI, offering insights into tissue homogeneity and potential tumor characterization. It can provide valuable information about the spatial relationships and patterns of pixel intensities within a region of interest (ROI), such as a brain tumor. Higher correlation values may indicate more homogeneous tissue structures, while lower values suggest heterogeneity, which can be associated with tumor aggressiveness. Variations in texture features like GLCM Correlation can assist in differentiating tumor types and grades, potentially aiding in diagnosis and treatment planning. Its interpretation requires careful consideration of imaging protocols, accurate tumor segmentation, and integration with other clinical data.
Pr

# Simple Method RAG

In [3]:
import os
from gpt4all import GPT4All

# Define the RetrievalSystem class
class RetrievalSystem:
    def __init__(self, file_paths):
        self.data = self.load_files(file_paths)

    def load_files(self, file_paths):
        data = []
        for file_path in file_paths:
            try:
                with open(file_path, 'r') as file:
                    data.append(file.read())
            except FileNotFoundError:
                print(f"Error: File not found - {file_path}")
        return data

    def retrieve(self, query):
        # Simple retrieval: return all content for demo purposes
        # In a real-world scenario, you would implement a more sophisticated retrieval method
        return ' '.join(self.data)

# Initialize the retrieval system with the paths to the .txt files
file_paths = ['file1.txt', 'file2.txt', 'file3.txt']
retrieval_system = RetrievalSystem(file_paths)

# Initialize the GPT-4 model
model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf")

# Function to generate diagnosis using RAG
def generate_diagnosis(query):
    # Retrieve relevant information
    retrieved_data = retrieval_system.retrieve(query)

    # Combine retrieved data with the query for context
    augmented_query = f"{query}\n{retrieved_data}"

    # Generate response using the model
    with model.chat_session():
        response = model.generate(augmented_query, max_tokens=210)

    return response

# Example usage
output = generate_diagnosis("What are the symptoms of brain tumors?")
print(output)

Error: File not found - file1.txt
Error: File not found - file2.txt
Error: File not found - file3.txt


Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7a60f781a5f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gpt4all/_pyllmodel.py", line 614, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <function LLModel._callback_decoder.<locals>._raw_callback at 0x7a60f7a888b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gpt4all/_pyllmodel.py", line 573, in _raw_callback
    def _raw_callback(token_id: int, response: bytes) -> bool:
KeyboardInterrupt: 


Brain tumors can cause a wide range of symptoms, depending on their location, size, and type


# Fine-Tune

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset

# Load the pre-trained model and tokenizer
model_name = "nomic-ai/gpt4all-j"  # Replace with your specific GPT4All model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Prepare your custom dataset
# This is a simplified example. Replace with your actual data.
train_data = [
    {"input": "Describe the symptoms of a brain tumor.", "output": "Common symptoms of brain tumors include headaches, seizures, vision problems, and cognitive changes."},
    {"input": "What is the significance of GLCM correlation in brain tumor analysis?", "output": "GLCM correlation in brain tumor analysis helps identify texture patterns in MRI scans, potentially distinguishing between tumor types and healthy tissue."},
    {"input": "Explain the importance of shape elongation in tumor characterization.", "output": "Shape elongation in tumor characterization quantifies how stretched a tumor is, which can indicate growth patterns and potential aggressiveness."},
    # Add more examples...
]

# Function to tokenize and format the data
def tokenize_function(examples):
    inputs = [f"Input: {item['input']}\nOutput: {item['output']}" for item in examples]
    return tokenizer(inputs, padding="max_length", truncation=True, max_length=512)

# Create a Hugging Face Dataset
dataset = Dataset.from_list(train_data)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_gpt4all")
tokenizer.save_pretrained("./fine_tuned_gpt4all")

# Function to generate responses using the fine-tuned model
def generate_response(input_text):
    input_ids = tokenizer.encode(f"Input: {input_text}\nOutput:", return_tensors="pt")
    output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example usage
print(generate_response("What are the key features used in brain tumor radiomics?"))